In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm

link_inicial="https://www.encuentra24.com"
link_venta_casas = "https://www.encuentra24.com/el-salvador-es/bienes-raices-venta-de-propiedades-casas"
payload = {
    "q":"number.50", # para tener 50 resultados por cada pagina, disminuye el bucle
}
r = requests.get(link_venta_casas, params=payload)

soup = BeautifulSoup(r.text, 'html.parser')

links = ["".join([link_inicial, link.attrs["href"].split("?")[0]]) for link in soup.find_all(name="div", attrs={"class":"ann-ad-tile__cover"})]

links = []
contador = 1
while True:
    link_temp = f"https://www.encuentra24.com/el-salvador-es/bienes-raices-venta-de-propiedades-casas.{contador}"
    payload = {
        "q":"number.50", # para tener 50 resultados por cada pagina, disminuye el bucle
    }

    r = requests.get(link_temp, params=payload)
    soup = BeautifulSoup(r.text, 'html.parser')
    links_temp = ["".join([link_inicial, link.attrs["href"].split("?")[0]]) for link in soup.find_all(name="div", attrs={"class":"ann-ad-tile__cover"})]
    if len(links_temp)==0:
        break
    links.extend(links_temp)

    contador+=1

In [84]:
def check_proyecto_nuevo(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text, "html.parser")
    try:
        breadcrumb = soup.find(name="div", attrs={"class":"d3-container adaptor-breadcrumb-detailpager"}).text
        saltarse_link = False
    except AttributeError:
        saltarse_link = True

    if breadcrumb.find("Proyectos nuevos")!=-1:
        return True, soup, r, saltarse_link
    else:
        return False, soup, r, saltarse_link

def scrape_proyecto_nuevo():
    pass

def scrape_proyecto_normal(soup, r):
    ad_info = soup.find(name="div", attrs={"class":"ad-info"})
    ad_info_parts = ad_info.find_all(name="li")

    info_dict = {}

    for part in ad_info_parts:
        info_name = part.find(name="span", attrs={"class":"info-name"}).text
        info_value = part.find(name="span", attrs={"class":"info-value"}).text
        info_dict[info_name] = [info_value]


    ad_details = soup.find(name="div", attrs={"class":"ad-details"})
    ad_details_part = ad_details.find_all(name="li")
    for part in ad_details_part:
        info_name = part.find(name="span", attrs={"class":"info-name"}).text
        info_value = part.find(name="span", attrs={"class":"info-value"}).text

        info_dict[info_name] = [info_value]


    section_box = soup.find_all(name="div", attrs={"class":"section-box"})
    section_box = [i for i in section_box if i.text.find("Detalles")!=-1][0]

    section_box_id = section_box.find(name="span", attrs={"class":"ad-id"}).text
    section_box_content_b = section_box.find(name=["b"]).text
    section_box_content_p = section_box.find(name=["p"]).text
    info_dict["Detalles"] = ["\n".join([section_box_content_b, section_box_content_p])]
    info_dict["id"] = [section_box_id]
    info_dict["link"] = [r.request.url]

    return pd.DataFrame(info_dict)

In [86]:
df_master = pd.DataFrame()

links_shortened = links.copy()
for contador_barra in tqdm(range(len(links_shortened))):
    for casa in links_shortened:
        is_proyecto_nuevo, soup, r, saltarse_link = check_proyecto_nuevo(casa)

        if saltarse_link:
            continue
        
        if is_proyecto_nuevo:
            scrape_proyecto_nuevo()
            continue
        else:        
            df_tmp = scrape_proyecto_normal(soup=soup, r=r)

        df_master = pd.concat([df_master, df_tmp])

  0%|          | 0/884 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [77]:
df_master

,Categoria:,Publicado:,Localización:,Precio:,Precio/M² de construcción:,Dirección exacta:,Costos de mantenimiento:,Piso Número:,Recámaras:,Baños:,...,link,Precio/M² de terreno:,Tamaño del lote:,Año de construcción:,Niveles:,Piscina:,Tipo de pisos:,Balcón/Terraza:,Altura:,PDF con info de la propiedad:
0,Casas,07/08/2023,Santa Tecla,"$975,000","$1,625",¡Pregunta al anunciante!,¡Pregunta al anunciante!,¡Pregunta al anunciante!,5,6,...,https://www.encuentra24.com/el-salvador-es/bie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Casas,04/08/2023,San Salvador,"$295,000","$1,353",¡Pregunta al anunciante!,¡Pregunta al anunciante!,¡Pregunta al anunciante!,3,3.5,...,https://www.encuentra24.com/el-salvador-es/bie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Casas,02/08/2023,San Salvador,"$498,000 (Rebajado 22%)\n","$24,900",¡Pregunta al anunciante!,¡Pregunta al anunciante!,¡Pregunta al anunciante!,6,7,...,https://www.encuentra24.com/el-salvador-es/bie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Casas,02/08/2023,Usulután,"$120,700","$1,022","Calle Mustang y final Calle Nathania, Costado ...",¡Pregunta al anunciante!,¡Pregunta al anunciante!,3,2,...,https://www.encuentra24.com/el-salvador-es/bie...,$609.59,198,2023,1,no,Porcelanato,NaN,NaN,NaN
0,Casas,01/08/2023,San Salvador,"$875,000","$1,944",¡Pregunta al anunciante!,¡Pregunta al anunciante!,¡Pregunta al anunciante!,4,3,...,https://www.encuentra24.com/el-salvador-es/bie...,"$1,011",865,NaN,NaN,NaN,NaN,terraza,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Casas,03/07/2023,Zaragoza,"$650,000","$1,625",¡Pregunta al anunciante!,¡Pregunta al anunciante!,¡Pregunta al anunciante!,3,3,...,https://www.encuentra24.com/el-salvador-es/bie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Casas,03/07/2023,San Salvador,"$465,000",$11.58,¡Pregunta al anunciante!,¡Pregunta al anunciante!,¡Pregunta al anunciante!,3,3.5,...,https://www.encuentra24.com/el-salvador-es/bie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Casas,03/07/2023,Santa Tecla,"$605,000","$1,983",¡Pregunta al anunciante!,¡Pregunta al anunciante!,¡Pregunta al anunciante!,3,3,...,https://www.encuentra24.com/el-salvador-es/bie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Casas,03/07/2023,Chalatenango,"$75,000",$500.00,¡Pregunta al anunciante!,¡Pregunta al anunciante!,¡Pregunta al anunciante!,3,2,...,https://www.encuentra24.com/el-salvador-es/bie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
df_master

,Categoria:,Publicado:,Localización:,Precio:,Precio/M² de construcción:,Dirección exacta:,Costos de mantenimiento:,Piso Número:,Recámaras:,Baños:,Parking:,M² de construcción:,Detalles,id,link
0,Casas,07/08/2023,Santa Tecla,"$975,000","$1,625",¡Pregunta al anunciante!,¡Pregunta al anunciante!,¡Pregunta al anunciante!,5,6,Más,600,Preciosa Residencia de lujo en Palmira\nPrecio...,ID.: 24575823,https://www.encuentra24.com/el-salvador-es/bie...
0,Casas,04/08/2023,San Salvador,"$295,000","$1,353",¡Pregunta al anunciante!,¡Pregunta al anunciante!,¡Pregunta al anunciante!,3,3.5,2,218,"Se vende casa moderna en Miramar, Casa de 1 pl...",ID.: 24181090,https://www.encuentra24.com/el-salvador-es/bie...
0,Casas,02/08/2023,San Salvador,"$498,000 (Rebajado 22%)\n","$24,900",¡Pregunta al anunciante!,¡Pregunta al anunciante!,¡Pregunta al anunciante!,6,7,1,20,Se vende y se alquila (Negociable)\nCasa de 3 ...,ID.: 24473602,https://www.encuentra24.com/el-salvador-es/bie...
